CPU-bound: If your task is computationally intensive (e.g., image filtering, resizing, or transformations), focus on multiprocessing.

I/O-bound: If the task involves reading/writing data to disk or the network (e.g., loading/saving images), focus on async I/O.


---

### **Key Comparison**

| **Feature**         | **Multiprocessing**              | **Asyncio**                    |
|----------------------|----------------------------------|--------------------------------|
| **Concurrency Type** | Parallelism (multiple cores)     | Cooperative multitasking       |
| **Best For**         | CPU-bound tasks                 | I/O-bound tasks                |
| **Memory Usage**     | High (separate processes)       | Low (single-threaded)          |
| **Performance**      | Scales with CPU cores           | Scales with number of I/O tasks |
| **Complexity**       | Higher (IPC required)           | Moderate (requires async/await) |
| **GIL Impact**       | Bypasses GIL                    | Limited by GIL                 |
| **Debugging**        | Can be complex                  | Moderate complexity            |

---

### **Which to Use?**
1. **Use Multiprocessing**:
   - When tasks are computation-heavy.
   - When you need to use multiple CPU cores for faster execution.

2. **Use Asyncio**:
   - When tasks involve heavy I/O (e.g., fetching data, network calls).
   - When you need to handle many simultaneous connections efficiently.

For hybrid workloads (CPU + I/O), you can combine both approaches. For example, use **multiprocessing** for parallel computations and **asyncio** for managing I/O tasks within each process.

ACTUAL CASE

1. Suppose I have task I fetch information from 3 server, I will implement: (I/O-bound)

---
```python
import asyncio
import aiohttp

async def fetch_data(session, url):
    try:
        async with session.get(url) as response:
            data = await response.text()  # You can change this to `.json()` if the API returns JSON.
            return data
    except Exception as e:
        print(f"Error fetching from {url}: {e}")
        return None

async def fetch_from_servers(servers):
    async with aiohttp.ClientSession() as session:
        # Start fetching from all servers concurrently
        tasks = [asyncio.create_task(fetch_data(session, url)) for url in servers]
        
        # Use asyncio's `as_completed` to process tasks as they finish
        for task in asyncio.as_completed(tasks):
            result = await task
            if result:  # If we got a valid result
                print("Got data:", result)
                # Cancel remaining tasks
                for t in tasks:
                    t.cancel()
                return result
        
        # If no server returned valid data
        print("No valid response from any server.")
        return None

# Example usage
servers = [
    "https://jsonplaceholder.typicode.com/posts/1",
    "https://jsonplaceholder.typicode.com/posts/2",
    "https://jsonplaceholder.typicode.com/posts/3",
]

asyncio.run(fetch_from_servers(servers))


```

---

2. Process data, use multiple process for speeding up (CPU-bound)

```python
import os
from glob import glob
from tqdm import tqdm
import multiprocessing as mp

# Example process function
def process(idx):
    # Read the image file using its index
    file_path = files[idx]
    # Perform some image processing task
    # For example, let's assume we are resizing the images using OpenCV
    import cv2
    img = cv2.imread(file_path)
    resized_img = cv2.resize(img, (256, 256))
    # Save or do something with the processed image
    output_path = file_path.replace('/images/', '/processed_images/')
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, resized_img)

# Main script
for folder in os.listdir('/mnt/md0/dacl10k/images/')[1:]:
    files = sorted(glob(f'/mnt/md0/dacl10k/images/{folder}/*.jpg'))
    
    start = 0
    with mp.Pool(processes=8) as pool:
        idxs = list(range(start, len(files)))
        imap = pool.imap(process, idxs)
        _ = list(tqdm(imap, total=len(files) - start))
```